In [7]:
from vnpy.trader.constant import (Exchange, Interval)
import pandas as pd
import os
import glob
import datetime

In [3]:
symbol_info_frame = pd.read_csv(r"E:\project\trade_engine\iqfeed_data_processor\futures_to_import.csv")

In [8]:
iqfeed_1min_data_folder = r"D:\data\iqfeed\future\future_data_1min_iqfeed"
vnpy_1min_data_folder = r"D:\data\iqfeed\future\future_data_1min_vnpy"

In [6]:
iqfeed_data_path_list = glob.glob(os.path.join(iqfeed_1min_data_folder, "*.csv"))

In [11]:
__YEARS__ = ["06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
__MONTH_CODES__ = ["F", "G", "H", "J", "K", "M", "N", "Q", "U", "V", "X", "Z"]

for ix, row in symbol_info_frame.iterrows():
    symbol_root = row.loc["symbol_roots"]
    symbol_ib_root = row.loc["symbol_ib_roots"]
    exchange = row.loc["symbol_exchange"]
    for year in __YEARS__:
        for month_code in __MONTH_CODES__:
            symbol = "".join([symbol_root, month_code, year])
            symbol_csv_string = "".join([symbol, "_1.csv"])
            symbol_csv_path = os.path.join(iqfeed_1min_data_folder, symbol_csv_string)
            ib_symbol = "".join([symbol_ib_root, month_code, year])
            if symbol_csv_path in iqfeed_data_path_list:
                imported_data = pd.read_csv(symbol_csv_path)
                imported_data.columns = ["datetime", "open", "high", "low", "close", "volume"]
                imported_data.loc[:, "exchange"] = Exchange(exchange)
                imported_data.loc[:, "open_interest"] = 0.0
                imported_data.loc[:, "interval"] = Interval.MINUTE
                imported_data.loc[:, "symbol"] = ib_symbol
                datetime_format = '%Y%m%d %H:%M:%S'
                imported_data['datetime'] = pd.to_datetime(imported_data['datetime'], format=datetime_format)
                float_columns = ['open', 'high', 'low', 'close', 'volume', 'open_interest']
                for col in float_columns:
                    imported_data.loc[:, col] = imported_data[col].astype('float')
                imported_data.to_csv(os.path.join(vnpy_1min_data_folder, ib_symbol + ".csv"), index=False)